In [86]:
# !pip install PPO
# !pip install stable-baselines3[extra]
# !pip install gym
# !pip install keras
# !pip install keras-rl2

# 1. Import Dependencies

In [107]:
import gym 
from gym import Env
from gym import spaces, logger
from gym.spaces import Discrete, Box, Dict, MultiBinary, MultiDiscrete 
import numpy as np
from numpy import linalg as LA
import random
from gym.utils import seeding
import os
from stable_baselines3 import PPO
from stable_baselines3 import A2C
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import math
#from google.colab import drive
# drive.mount('/content/drive')

# models_dir = "C:/Users/FARIS SYAHMI/Desktop/Kerja/5GReinforceLearning/5NRmodel_logs/model/PPO"
# logdir = "C:/Users/FARIS SYAHMI/Desktop/Kerja/5GReinforceLearning/5NRmodel_logs/logs"

# models_dir = "/content/drive/MyDrive/Colab Notebooks/5GNRmodel_logs/models/PPO"
# logdir = "/content/drive/MyDrive/Colab Notebooks/5GNRmodel_logs/logs"

models_dir = "ML Rl 5g31/model/PPO"
logdir = "ML Rl 5g31/logs"

if not os.path.exists(models_dir):
     os.makedirs(models_dir)

if not os.path.exists(logdir):
     os.makedirs(logdir)

## 2. Building an Environment

In [88]:
### unit conversion

def watts_dbW(power):
    dbW = 10*np.log10(power)
    return dbW

def dbW_dbm(dbW):
    dbm = dbW + 30
    return dbm

def dbW_watts(dbW):
    power = 10**(dbW/10)
    return power

def dbm_dbW(dbm):
    dbW = dbm - 30
    return dbW


In [156]:
class environment(Env):

########### init method or constructor #############################
    def __init__(self):
        # env config
        self.M_ULA = 16
        self.G_ant_no_beamforming = 11 # dBi        
        self.np_random = None
        self.absDistanceBS1 = 0
        self.absDistanceBS2 = 0
        self.UEtrans = 0
        self.UEdemand = 0
        self.UEpacket = 0
        self.ULDLratio = 0.9   #Uplink ratio
        self.ULDLratio2 = 0.9   #Uplink ratio
        self.ULDLratio3 = 0.9   #Uplink ratio
        self.UE = 100
        self.max_tx_power_interference = 10/2 #in watts
        self.subcarrierSpacing1 = 4
        self.subcarrierSpacing2 = 4
        self.subcarrierSpacing3 = 4
        self.maxTxPower = 60
        self.txPower = 45 #watt
        self.UEtxPower = dbW_watts(43-30)   ##43bdm to watt
        self.f_c = 700e6  # Hz
        self.cell_radius = 150  # in meters.
        self.inter_site_distance =  self.cell_radius / 2.
        self.FSPL = False
        self.x_bs_1, self.y_bs_1 = 0, 0
        self.x_bs_2, self.y_bs_2 = self.inter_site_distance+self.cell_radius, 0
        self.guardband1 = 4
        self.guardband2 = 4
        self.use_beamforming = False
        self.Np = 4 # from 3 to 5 for mmWave
        self.prob_LOS = 0.8 # Probability of LOS transmission        
        # RL config
        self.seed(seed=10)
        self.state = None
        self.num_actions = 511
        self.step_count = 10 # which step
        self.reward_min = -10000000
        self.reward_max = 10000
        self.length = 1000

        bounds_lower = np.array([
            2,
            dbW_watts(43-30),
            0.1,
            0.1,
            0.1,
            5,
            self.txPower,
            self.max_tx_power_interference,
            0,
            0,
            1,
            1,
            1,
            ])

        bounds_upper = np.array([
            self.UE,
            self.UEtxPower,
            self.ULDLratio,
            self.ULDLratio2,
            self.ULDLratio3,                     
            self.inter_site_distance + self.cell_radius,
            self.txPower,
            self.max_tx_power_interference,
            self.guardband1, 
            self.guardband2, 
            self.subcarrierSpacing1,
            self.subcarrierSpacing2,
            self.subcarrierSpacing3,       
            ])

        self.action_space = spaces.Discrete(self.num_actions)  # action size is here
        self.observation_space = spaces.Box(bounds_lower, bounds_upper)
                                            #, dtype=np.int)
                                            #,shape=(7,),
                                            #dtype=np.float32)  # spaces.Discrete(2) # state size is here 
        int_ue,int_ueTx,int_ratio1,int_ratio2,int_ratio3,int_radius,int_tx,int_itx,int_gb,int_gb2,int_ss1,int_ss2,int_ss3 = self.observation_space.sample()                                    
        # self.intSINR = self.SINR(int_tx,int_itx,int_ratio,int_gb,int_ss1,int_ss2)                                    
        # self.intBitrate = self.bitrate(int_ratio,int_ss1)
        self.intBitrate = 10
        self.intSINR = dbW_watts(40) #db to watt
        # Set start state
        self.state = None
        # Set shower length


########### seeding #############################

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]        

########### step action #############################

    def step(self, action):
        

        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        (UE,UEtxpower,ULDLratio,ULDLratio2,ULDLratio3, x_cell_radius, txPower, interferencePower, guardband, guardband2, subcarrierSpacing1, subcarrierSpacing2,subcarrierSpacing3) = self.state
        #print(action)
        # power_tx = action & 0b00000001            # 0 power up, 1 power down
        ratio = (action & 0b0000001)             
        subcarrier1 = (action & 0b0000010)       >> 1
        gb = (action & 0b00001000)                >> 2
        subcarrier2 = (action & 0b0001000)       >> 3
        subcarrier3 = (action & 0b00100000)       >> 4 
        ratio2 = (action & 0b0001000000)             >> 5
        ratio3 = (action & 0b0010000000)             >> 6
        gb2 = (action & 0b010000000)                >> 7
        subcarrier2 = (action & 0b0100000000)       >> 8
        subcarrier3 = (action & 0b1000000000)       >> 9        
        UE = math.ceil(UE)

 

        # if (power_tx == 1 and txPower > 1):
        #     txPower *= 10**(0.1/10.)
        # elif (power_tx == 0 and txPower < 50):
        #     txPower *= 10**(-0.1/10.)

        #on if want tx to change    
        # if (power_tx == 0 and txPower > 5 and txPower < 50):
        #     txPower += 2
        # elif (power_tx == 1 and txPower > 5 and txPower < 50):
        #     txPower -= 2 

        if (ratio == 1 and ULDLratio>=0.1 and ULDLratio<=0.9):
            ULDLratio  = ULDLratio + 0.05
        elif (ratio == 0 and ULDLratio>=0.1 and ULDLratio<=0.9):
            ULDLratio = ULDLratio - 0.05
        
        if (ratio2 == 1 and ULDLratio2>=0.1 and ULDLratio2<=0.9):
            ULDLratio2  = ULDLratio2 + 0.05
        elif (ratio2 == 0 and ULDLratio2>=0.1 and ULDLratio2<=0.9):
            ULDLratio2 = ULDLratio2 - 0.05
        
        if (ratio3 == 1 and ULDLratio3>=0.1 and ULDLratio3<=0.9):
            ULDLratio3  = ULDLratio3 + 0.05
        elif (ratio3 == 0 and ULDLratio3>=0.1 and ULDLratio3<=0.9):
            ULDLratio3 = ULDLratio3 - 0.05      
        # print(ULDLratio)
        # print(ULDLratio2)
        # print(ULDLratio3)

        if (subcarrier1 == 1 and subcarrierSpacing1>=1 and subcarrierSpacing1<=3):
            subcarrierSpacing1 = subcarrierSpacing1 + 1
        elif(subcarrier1 == 0 and subcarrierSpacing1>=2 and subcarrierSpacing1<=4):
            subcarrierSpacing1 = subcarrierSpacing1 - 1

        if (subcarrier2 == 1 and subcarrierSpacing2>=1 and subcarrierSpacing2<=3):
            subcarrierSpacing2 = subcarrierSpacing2 + 1
        elif(subcarrier2 == 0 and subcarrierSpacing2>=2 and subcarrierSpacing2<=4):
            subcarrierSpacing2 = subcarrierSpacing2 - 1

        if (subcarrier3 == 1  and subcarrierSpacing3>=1 and subcarrierSpacing3<=3):
            subcarrierSpacing3 = subcarrierSpacing3 + 1
        elif(subcarrier3 == 0 and subcarrierSpacing3>=2 and subcarrierSpacing3<=4):
            subcarrierSpacing3 = subcarrierSpacing3 - 1                 

        if (gb == 1  and guardband>=0 and guardband<=3):
            guardband = guardband + 1
        elif (gb == 0  and guardband>=1 and guardband<=4):
            guardband = guardband - 1

        if (gb2 == 1 and guardband2>=0 and guardband2<=3):
            guardband2 = guardband2 + 1
        elif (gb2 == 0 and guardband2>=1 and guardband2<=4):
            guardband2 = guardband2 - 1

        guardband1 = [guardband,guardband2]
        guardband_f = [guardband,0]    
        guardband_b = [guardband2,0] 
                                

        
        # Reduce shower length by 1 second
         
        
        # print(self.length)
        

        #divide UE into link based on demand
        if (self.length == 1000):
            self.absDistanceBS1,self.absDistanceBS2,self.UEtrans,self.UEdemand,self.UEpacket= self.UEposition(UE)
            # a,b,c,d,e= self.UEposition(UE)
            # self.store(absDistanceBS1,absDistanceBS2,UEtrans,UEdemand,UEpacket)
            # absDistanceBS1 = a
            # absDistanceBS2 = b
            # UEtrans = c
            # UEdemand = d
            # UEpacket = e
        else:
            None


        self.length -= 1
        minpower = self.minpower(UE,self.absDistanceBS1)
        maxpower = self.txPower
        Ue_mmtc_UL = 0 
        Ue_EMBB_UL = 0
        Ue_URLLC_UL = 0 
        Ue_mmtc_DL = 0
        Ue_EMBB_DL = 0
        Ue_URLLC_DL = 0
        mm_D_bs1_UL = []
        mm_D_bs2_UL =[]
        mm_D_bs1 = []
        mm_D_bs2 =[]
        ur_D_bs1_UL = []
        ur_D_bs2_UL =[]
        ur_D_bs1 = []
        ur_D_bs2 =[]
        em_D_bs1_UL = []  
        em_D_bs2_UL =[]
        em_D_bs1 = []
        em_D_bs2 =[]
        listReward = []
        number = 0
        used = 0
        reward1 = 0
        reward2 = 0
        rewardEMBB = 0
        rewardmmtc = 0
        rewardURLLC = 0
        reward = 0
        rewardPowerMin = 0
        rewardPowerMax = 0

        

#         for i in range(0,UE):
#             if UEdemand[i] <= 20:
#                 if UEtrans[i] == 0:
#                     mm_D_bs1_UL.append(Ue_mmtc_UL)
#                     mm_D_bs2_UL.append(Ue_mmtc_UL)
#                     mm_D_bs1_UL[Ue_mmtc_UL] = absDistanceBS1[i]
#                     mm_D_bs2_UL[Ue_mmtc_UL] = absDistanceBS2[i]
#                     Ue_mmtc_UL += 1

#                 else:     
#                     mm_D_bs1.append(Ue_mmtc_DL)
#                     mm_D_bs2.append(Ue_mmtc_DL)
#                     mm_D_bs1[Ue_mmtc_DL] = absDistanceBS1[i]
#                     mm_D_bs2[Ue_mmtc_DL] = absDistanceBS2[i]
#                     Ue_mmtc_DL += 1

#             elif UEdemand[i]>20:
#                 if UEtrans[i] == 0:
#                     ur_D_bs1_UL.append(Ue_URLLC_UL)
#                     ur_D_bs2_UL.append(Ue_URLLC_UL)
#                     ur_D_bs1_UL[Ue_URLLC_UL] = absDistanceBS1[i]
#                     ur_D_bs2_UL[Ue_URLLC_UL] = absDistanceBS2[i]
#                     Ue_URLLC_UL += 1

#                 else:
#                     ur_D_bs1.append(Ue_URLLC_DL)
#                     ur_D_bs2.append(Ue_URLLC_DL)
#                     ur_D_bs1[Ue_URLLC_DL] = absDistanceBS1[i]
#                     ur_D_bs2[Ue_URLLC_DL] = absDistanceBS2[i]
#                     Ue_URLLC_DL += 1

#             elif UEdemand[i]>50:
#                 if UEtrans[i] == 0:
#                     em_D_bs1_UL.append(Ue_EMBB_UL)
#                     em_D_bs2_UL.append(Ue_EMBB_UL)      
#                     em_D_bs1_UL[Ue_EMBB_UL] = absDistanceBS1[i]
#                     em_D_bs2_UL[Ue_EMBB_UL] = absDistanceBS2[i]
#                     Ue_EMBB_UL += 1

#                 else:
#                     em_D_bs1.append(Ue_EMBB_DL)
#                     em_D_bs2.append(Ue_EMBB_DL)
#                     em_D_bs1[Ue_EMBB_DL] = absDistanceBS1[i]
#                     em_D_bs2[Ue_EMBB_DL] = absDistanceBS2[i]
#                     Ue_EMBB_DL += 1    

        for i in range(0,UE):
            if self.UEpacket[i] <= 20:
                if self.UEtrans[i] == 0:
                    mm_D_bs1_UL.append(Ue_mmtc_UL)
                    mm_D_bs2_UL.append(Ue_mmtc_UL)
                    mm_D_bs1_UL[Ue_mmtc_UL] = self.absDistanceBS1[i]
                    mm_D_bs2_UL[Ue_mmtc_UL] = self.absDistanceBS2[i]
                    Ue_mmtc_UL += 1

                else:     
                    mm_D_bs1.append(Ue_mmtc_DL)
                    mm_D_bs2.append(Ue_mmtc_DL)
                    mm_D_bs1[Ue_mmtc_DL] = self.absDistanceBS1[i]
                    mm_D_bs2[Ue_mmtc_DL] = self.absDistanceBS2[i]
                    Ue_mmtc_DL += 1

            elif self.UEpacket[i]>20 and self.UEpacket[i] < 99:
                if self.UEtrans[i] == 0:
                    ur_D_bs1_UL.append(Ue_URLLC_UL)
                    ur_D_bs2_UL.append(Ue_URLLC_UL)
                    ur_D_bs1_UL[Ue_URLLC_UL] = self.absDistanceBS1[i]
                    ur_D_bs2_UL[Ue_URLLC_UL] = self.absDistanceBS2[i]
                    Ue_URLLC_UL += 1

                else:
                    ur_D_bs1.append(Ue_URLLC_DL)
                    ur_D_bs2.append(Ue_URLLC_DL)
                    ur_D_bs1[Ue_URLLC_DL] = self.absDistanceBS1[i]
                    ur_D_bs2[Ue_URLLC_DL] = self.absDistanceBS2[i]
                    Ue_URLLC_DL += 1

            elif self.UEpacket[i]>100:
                if self.UEtrans[i] == 0:
                    em_D_bs1_UL.append(Ue_EMBB_UL)
                    em_D_bs2_UL.append(Ue_EMBB_UL)      
                    em_D_bs1_UL[Ue_EMBB_UL] = self.absDistanceBS1[i]
                    em_D_bs2_UL[Ue_EMBB_UL] = self.absDistanceBS2[i]
                    Ue_EMBB_UL += 1

                else:
                    em_D_bs1.append(Ue_EMBB_DL)
                    em_D_bs2.append(Ue_EMBB_DL)
                    em_D_bs1[Ue_EMBB_DL] = self.absDistanceBS1[i]
                    em_D_bs2[Ue_EMBB_DL] = self.absDistanceBS2[i]
                    Ue_EMBB_DL += 1   
        
        
        #track number of user each link
        if (self.length == 1000-1):
            print("Ue_mmtc_UL = " + str(Ue_mmtc_UL))
            print("Ue_EMBB_UL = " + str(Ue_EMBB_UL))
            print("Ue_URLLC_UL = " + str(Ue_URLLC_UL))
            print("Ue_mmtc_DL = " + str(Ue_mmtc_DL))
            print("Ue_EMBB_DL = " + str(Ue_EMBB_DL))
            print("Ue_URLLC_DL = " + str(Ue_URLLC_DL))
        else:
            None    
        

          # Calculate reward link 1 URLLC
        SINRupdated1_UL, SEupdated1_UL,Throughput_updated1_UL= self.SINR(Ue_URLLC_UL,ur_D_bs1_UL,ur_D_bs2_UL,UEtxpower,interferencePower,ULDLratio,guardband_f,subcarrierSpacing1,subcarrierSpacing2,0,"UL")
        self.intBitrate1 = self.bitrate(ULDLratio, subcarrierSpacing1)
        #print("SINRupdated1_UL" + str(SINRupdated1_UL))
        # print("Throughput_updated1_UL" + str(Throughput_updated1_UL))
          


          # Calculate reward link 2 EMBB
        SINRupdated2_UL, SEupdated2_UL,Throughput_updated2_UL= self.SINR(Ue_EMBB_UL,em_D_bs1_UL,em_D_bs2_UL,UEtxpower,interferencePower,ULDLratio2,guardband1,0,subcarrierSpacing2,subcarrierSpacing3,"UL")
        self.intBitrate2 = self.bitrate(ULDLratio, subcarrierSpacing2)
        #print("SINRupdated2_UL" + str(SINRupdated2_UL))
        # print("Throughput_updated2_UL" + str(Throughput_updated2_UL))
        
          # Calculate reward link 3 MMTC
          
        SINRupdated3_UL, SEupdated3_UL,Throughput_updated3_UL= self.SINR(Ue_mmtc_UL,mm_D_bs1_UL,mm_D_bs2_UL,UEtxpower,interferencePower,ULDLratio3,guardband_b,subcarrierSpacing1,subcarrierSpacing2,subcarrierSpacing3,"UL")
        self.intBitrate3 = self.bitrate(ULDLratio, subcarrierSpacing3)
        #print("SINRupdated3_UL" + str(SINRupdated3_UL))
        # print("Throughput_updated3_UL" + str(Throughput_updated3_UL))

          #downlink reward

        SINRupdated1_DL, SEupdated1_DL,Throughput_updated1_DL= self.SINR(Ue_URLLC_DL,ur_D_bs1,ur_D_bs2,txPower,interferencePower,1-ULDLratio,guardband_f,subcarrierSpacing1,subcarrierSpacing2,0,"DL")
        intBitrate1 = self.bitrate(1-ULDLratio, subcarrierSpacing1)
        #print("SINRupdated1_DL" + str(SINRupdated1_DL))
        # print("Throughput_updated1_DL" + str(Throughput_updated1_DL))


          # Calculate reward link 2 EMBB
        SINRupdated2_DL, SEupdated2_DL,Throughput_updated2_DL= self.SINR(Ue_EMBB_DL,em_D_bs1,em_D_bs2,txPower,interferencePower,1-ULDLratio2,guardband1,0,subcarrierSpacing2,subcarrierSpacing3,"DL")
        intBitrate2 = self.bitrate(1-ULDLratio2, subcarrierSpacing2)
        #print("SINRupdated2_DL" + str(SINRupdated2_DL))
        # print("Throughput_updated2_DL" + str(Throughput_updated2_DL))


          # Calculate reward link 3 MMTC

        SINRupdated3_DL, SEupdated3_DL,Throughput_updated3_DL= self.SINR(Ue_mmtc_DL,mm_D_bs1,mm_D_bs2,txPower,interferencePower,1-ULDLratio3,guardband_b,subcarrierSpacing1,subcarrierSpacing2,subcarrierSpacing3,"DL")
        intBitrate3 = self.bitrate(1-ULDLratio3, subcarrierSpacing3)
        #print("SINRupdated3_DL" + str(SINRupdated3_DL))
        # print("Throughput_updated3_DL" + str(Throughput_updated3_DL))
        
        # numberUE = Ue_mmtc_UL + Ue_EMBB_UL + Ue_URLLC_UL + Ue_mmtc_DL + Ue_EMBB_DL + Ue_URLLC_DL 
        # for i in range (numberUE)
        #     t_maxpower[i] = self.maxpower
        #     total = total + t_maxpower[i]

        listReward = [SINRupdated1_DL,SINRupdated2_DL,SINRupdated3_DL,SINRupdated1_UL,SINRupdated2_UL,SINRupdated3_UL]
        for i in range (0,6,1):
            if listReward[i] == 0:
                number+=1
        used = 6-number
        
        if((Ue_mmtc_UL or Ue_mmtc_DL) != 0 ):
            if ((Ue_mmtc_UL > Ue_mmtc_DL and Throughput_updated3_UL > Throughput_updated3_DL)  or (Ue_mmtc_UL < Ue_mmtc_DL and Throughput_updated3_UL < Throughput_updated3_DL)):
                rewardmmtc = 10000
                # print("mmtctrue")
            else:    
                rewardmmtc = -10000
                # print("mmtcfalse")
        else:        
            rewardmmtc = 10000
            # print("mmtctrue02")


        if((Ue_EMBB_UL or Ue_EMBB_DL) != 0 ):
            if ((Ue_EMBB_UL > Ue_EMBB_DL and Throughput_updated2_UL > Throughput_updated2_DL) or (Ue_EMBB_UL < Ue_EMBB_DL and Throughput_updated2_UL < Throughput_updated2_DL)):
                rewardEMBB = 10000
                # print("embbtrue")
            else:    
                rewardEMBB = -10000 
                # print("embbfalse")   
        else:        
            rewardEMBB = 10000
            # print("EMBBtrue02")            

        if((Ue_URLLC_UL or Ue_URLLC_DL) != 0 ):
            if ((Ue_URLLC_UL > Ue_URLLC_DL and Throughput_updated1_UL > Throughput_updated1_DL) or (Ue_URLLC_UL < Ue_URLLC_DL and Throughput_updated1_UL < Throughput_updated1_DL) ):
                rewardURLLC = 10000
                # print("URLLCtrue")
            else:    
                rewardURLLC = -10000
                # print("URLLCfalse") 
        else:        
            rewardURLLC = 10000
            # print("URLLCtrue02")                   
        
        # if txPower>minpower:
        #     rewardPowerMin = 10000
        #     #print("trueMin")
        # else:
        #     rewardPowerMin = -10000
            
        # if maxpower>txPower:
        #     rewardPowerMax = 10000
        #     #print("trueMax")
        # else:
        #     rewardPowerMax = -10000   
            
            
        reward1 = ((SINRupdated1_DL + SINRupdated2_DL + SINRupdated3_DL + SINRupdated1_UL + SINRupdated2_UL + SINRupdated3_UL)/used)
        reward2 = rewardmmtc + rewardURLLC + rewardEMBB # + rewardPowerMin + rewardPowerMax
        # print("reward2 = "+ str(reward2))
        # print("rewardmmtc = "+ str(rewardmmtc))
        # print("rewardURLLC = "+ str(rewardURLLC))
        # print("rewardEMBB = "+ str(rewardEMBB))
        # print("rewardPowerMin = "+ str(rewardPowerMin))
        # print("rewardPowerMax = "+ str(rewardPowerMax))
        
        reward = reward1 + reward2
        # print("reward = "+ str(reward))

        # done = (txPower > minpower) and (txPower >= 0) and \
        #      ((Ue_URLLC_UL > Ue_URLLC_DL and Throughput_updated1_UL > Throughput_updated1_DL) or (Ue_URLLC_UL < Ue_URLLC_DL and Throughput_updated1_UL < Throughput_updated1_DL)) and \
        #      ((Ue_mmtc_UL > Ue_mmtc_DL and Throughput_updated3_UL > Throughput_updated3_DL) or (Ue_mmtc_UL < Ue_mmtc_DL and Throughput_updated3_UL < Throughput_updated3_DL)) and \
        #      ((Ue_EMBB_UL > Ue_EMBB_DL and Throughput_updated2_UL > Throughput_updated2_DL) or (Ue_EMBB_UL < Ue_EMBB_DL and Throughput_updated2_UL < Throughput_updated2_DL))   
            
        # abort = (txPower > self.maxTxPower) or (txPower < minpower) 

        # if abort == True:
        #     done = False
        #     reward = self.reward_min
        # elif done:
        #     reward += self.reward_max


        if (self.length <= 0): 
            done = True
        else:
            done = False
            # print(self.length)

        self.state = (UE,UEtxpower,ULDLratio,ULDLratio2,ULDLratio3, x_cell_radius, txPower, interferencePower, guardband, guardband2, subcarrierSpacing1, subcarrierSpacing2,subcarrierSpacing3)           

        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        self.state = (UE, UEtxpower, ULDLratio,ULDLratio2,ULDLratio3, x_cell_radius, txPower, interferencePower, guardband, guardband2, subcarrierSpacing1,subcarrierSpacing2,subcarrierSpacing3)
        # Return step information
        return np.array(self.state), reward, done, info

########### rendering #############################

    def render(self):
        # Implement viz
        pass

########### reset #############################
    def test(self):
        UE = 15.69694
        UEpwr = 13
        ULDL = 0.5
        ULDL2 = 0.5
        ULDL3 = 0.5
        d = 225
        tx = 35
        intP = 20
        g1 = 0
        g2 = 0
        s1 = 1
        s2 = 1
        s3 = 1


        li = [UE,UEpwr,ULDL,ULDL2,ULDL3,d,tx,intP,g1,g2,s1,s2,s3,]
        variable = np.array(li)
        print(variable)
        return variable

    def reset(self):
        # Initialize f_n of both cells
        self.state = [self.np_random.uniform(low=10,     high=self.UE),
                      self.np_random.uniform(low=1,     high=self.UEtxPower),
                      self.np_random.uniform(low=0.1,   high=self.ULDLratio),
                      self.np_random.uniform(low=0.1,   high=self.ULDLratio2),
                      self.np_random.uniform(low=0.1,   high=self.ULDLratio3),
                      self.np_random.uniform(low=0,   high=self.inter_site_distance + self.cell_radius),
                      self.np_random.uniform(low=45,   high=self.txPower),
                      self.np_random.uniform(low=1,   high=self.max_tx_power_interference / 2),
                      self.np_random.uniform(low=0,   high=self.guardband1),
                      self.np_random.uniform(low=0,   high=self.guardband2),
                      self.np_random.uniform(low=1,   high=self.subcarrierSpacing1),
                      self.np_random.uniform(low=1,   high=self.subcarrierSpacing2),
                      self.np_random.uniform(low=1,   high=self.subcarrierSpacing3),
                      ]
#         UE = 30
#         UEpwr = 13
#         ULDL = 0.5
#         ULDL2 = 0.5
#         ULDL3 = 0.5
#         d = 225
#         tx = 35
#         intP = 20
#         g1 = 0
#         g2 = 0
#         s1 = 1
#         s2 = 1
#         s3 = 1

        
#         self.state = [UE,UEpwr,ULDL,ULDL2,ULDL3,d,tx,intP,g1,g2,s1,s2,s3,]
        
        self.length = 1000
        return np.array(self.state)
        #return np.array(li)

    def store(self,absDistanceBS1,absDistanceBS2,UEtrans,UEdemand,UEpacket):
        a = absDistanceBS1
        b = absDistanceBS2
        c = UEtrans
        d = UEdemand
        e = UEpacket

        return a,b,c,d,e

########### calculation #############################
    def SINR(self,UE,distanceBS1,distanceBS2,txpower,intpower,ratio,gb,ss1,ss2,ss3,ULDL):
        direction = ULDL
        #print(ULDL)
        gb1,gb2 = gb
        T = 290  # Kelvins
        B = 20000  # Hz
        k_Boltzmann = 1.38e-23
        noisePower = k_Boltzmann * T * B  # this is in Watts
        UESinr = []
        UESinrW= []
        receivedSinr = 0
        # print("UE "+ str(UE))
        
        if UE == 0:
            UESinr = 0
            Throughput = 0
            SE = 0
            avgR_SINR = 0
        else:  
            for i in range(0,UE,1):  #UE    
                receivedSinr = self.receivedPower(txpower,distanceBS1[i]) / (self.receivedPower(intpower,distanceBS2[i]) + noisePower + self.iniPower(gb1,ss1,ss2,self.receivedPower(txpower,distanceBS1[i])) + self.iniPower(gb2,ss2,ss3,self.receivedPower(txpower,distanceBS1[i])))
                W_receivedSinr = watts_dbW(receivedSinr)
                # print("distance "+ str(distanceBS1[i]))
                # print("txPower "+ str(txpower))
                # print("receivedPower "+ str(self.receivedPower(txpower,distanceBS1[i])))
                # print("intPower "+ str(self.receivedPower(intpower,distanceBS2[i])))
                # print("noisePower "+ str(noisePower))
                # print("iniPower "+ str(self.iniPower(gb1,ss1,ss2,self.receivedPower(txpower,distanceBS1[i]))))
                # print("ini2Power "+ str(self.iniPower(gb2,ss2,ss3,self.receivedPower(txpower,distanceBS1[i]))))
                # print("W_receivedSinr "+str(W_receivedSinr))
                UESinr.append(i)
                UESinr[i] = W_receivedSinr
                UESinrW.append(i)
                UESinrW[i] = receivedSinr
                #print('receivedSinr Power = ' + str(receivedSinr))
            avgR_SINRW = np.mean(UESinrW)
            avgR_SINR = np.mean(UESinr)
            # print(txpower)
            # #print(avgR_SINRW)
            # print(avgR_SINR)
            # print("")
            SE = np.log2(1 + avgR_SINRW)
            Throughput = B*np.log2(1+avgR_SINRW)*ratio
        #print('Spectrum Efficiency = ' + str(SE))

        return avgR_SINR,SE,Throughput


    def UEposition(self,numUE):
        radiusBS1 = 200
        heightBS = 80
        intersite = radiusBS1/2 + radiusBS1
        xbs1,ybs1 = 0,0
        xbs2,ybs2 = intersite,0
        distanceBS1 = []
        distanceBS2 = []
        absDistanceBS1 =[]
        absDistanceBS2 =[]
        UEtrans=[]
        UEdemand=[]
        UEpacket=[]
        x=[]
        y=[]
        numUE = math.ceil(numUE)

        for i in range(0,numUE,1):
            x.append(i)
            y.append(i)
            distanceBS1.append(i)
            distanceBS2.append(i)
            absDistanceBS1.append(i)
            absDistanceBS2.append(i)
            UEtrans.append(i)
            UEdemand.append(i)
            UEpacket.append(i)
          
            x[i] = random.randint(-radiusBS1,intersite+radiusBS1)
            y[i] = random.randint(-radiusBS1,radiusBS1)
            distanceBS1[i] = math.sqrt((x[i]-xbs1)**2 + (y[i]-ybs1)**2)
            distanceBS2[i] = math.sqrt((x[i]-xbs2)**2 + (y[i]-ybs2)**2)
            absDistanceBS1[i] = math.sqrt(distanceBS1[i]**2 + (heightBS)**2)
            absDistanceBS2[i] = math.sqrt(distanceBS2[i]**2 + (heightBS)**2)
            #insert fix distance here
            UEtrans[i] = random.randint(0,1)
            UEdemand[i] = random.randint(10,100) #mbps
            UEpacket[i] = random.randint(1,150)
        return absDistanceBS1,absDistanceBS2,UEtrans,UEdemand,UEpacket

    def receivedPower(self,txpower,distance):
        d_mainBS = distance
        txpower_db =  watts_dbW(txpower)
        txpower_dbm = dbW_dbm(txpower_db)

        if self.FSPL == True:
            pathloss = 20 * np.log10(d_mainBS) + 20 * np.log10(self.f_c) + 32.44 #in db

        else:
            f_c = self.f_c
            c = 3e8  # speed of light
            d = d_mainBS
            h_B = 20
            h_R = 1.5

            # COST231
            C = 3
            a = (1.1 * np.log10(f_c / 1e6) - 0.7) * h_R - (1.56 * np.log10(f_c / 1e6) - 0.8)
            pathloss = 46.3 + 33.9 * np.log10(f_c / 1e6) + 13.82 * np.log10(h_B) - a + (
                    44.9 - 6.55 * np.log10(h_B)) * np.log10(d / 1000.) + C # in db
            #print("PL" + str(pathloss))        

        
        # Now the channel h, which is a vector in beamforming.
        # This computes the channel for user in serving BS from the serving BS.
        h_1 = self._compute_channel(distance)            
        receivedPower_LA = txpower * LA.norm(h_1, ord=2) ** 2
        receivedPower_db = txpower_db - pathloss 
        receivedPower =  dbW_watts(receivedPower_db) 
        # print("receivedPower" + str(receivedPower) )
        # print("receivedPower_LA" + str(receivedPower_LA) )
        #print("RP" + str(receivedPower))
        return receivedPower

    def bitrate(self,ratio,ss1):
        N_prb = 100
        ss = round(ss1)

        if ss == 1:
            N_prb = 270
        elif ss == 2:
            N_prb = 133
        elif ss == 3:
            N_prb = 65
        elif ss == 4:
            N_prb = 32


        J = 1  # carrier aggregation
        v = 1  # MIMO layers
        Q = 6  # modulation order
        Rmax = 0.92578125  # LDPC code
        f = 1  # scaling factor
        OH = 0.14  # overhead control
        ULDLratio = ratio
        T_ofdm = 0.0000357142857142852  # (10 ** -3) / (14 * 2^u)
        _bitrate = v * Q * f * Rmax * (12 * N_prb / T_ofdm) * (1 - OH)
        Mbitrate = _bitrate * (10 ** -6)
        Mbitrate = Mbitrate * ULDLratio

        #print('Max BitRate = ' + str(Mbitrate))
        return Mbitrate


    def iniPower(self,guard,ss1,ss2,receivedPowerInt):
        total = 0
        gb = 0
        guardtf = 0
        # print(ss2)
        # print(ss1)
        ss1 = round(ss1)
        ss2 = round(ss2)
        # print(ss2)
        # print(ss1)
        ini_power = []
        inipower = 0
        z1 = 240  
        non = 0     
        if (ss1 == 1):
            Tn_bs1 = 1025
        elif (ss1 == 2):
            Tn_bs1 = 512
        elif (ss1 == 3):
            Tn_bs1 = 256
        elif (ss1 == 4):
            Tn_bs1 = 128
        else:
            non = 1 

        if (ss2 == 1):
            Tn_bs2 = 1052
        elif (ss2 == 2):
            Tn_bs2 = 512
        elif (ss2 == 3):
            Tn_bs2 = 256
        elif (ss2 == 4):
            Tn_bs2 = 128
        else:
            non = 1 

        if (guard == 1):
            guardtf = 1052
        elif (guard == 2):
            guardtf = 512
        elif (guard == 3):
            guardtf = 256
        elif (guard == 4):
            guardtf = 128
        else:
            non = 1     

        
        if (ss1==ss2 or ss1==0 or ss2==0):
            inipower = 0
        else:
              for i in range(1,z1):
                gb = guardtf
                Tn_bs1 = 1025
                Tn_bs2 = 512
                n_cp = 1-0.07
                n_bs1 = round(Tn_bs1*n_cp) 
                n_bs2 = round(Tn_bs2*n_cp)         
                pt_bs2 = receivedPowerInt
                k = 100
                n2 = 2

                v1 = 0
                OFDM_overlap = n_bs1 / Tn_bs2
                gain = 1
                total_v = 0
                total_z = 0

                # a = pt_bs2 / n_bs2
                # b = gain / (n_bs2 * n_bs1)
                # c = np.sin(math.pi / n_bs2 * (z1 - v1 - gb) * OFDM_overlap * Tn_bs1)
                # d = np.sin(math.pi / n_bs2 * (z1 - v1 - gb))
                # e = np.absolute((c / d) ** 2)

                # g = np.sin(math.pi / n_bs2 * (z1 - v1 - gb)*Tn_bs1)  
                # i = np.absolute((g / d) ** 2)
                # j = OFDM_overlap * i

                if ss1 > ss2:
                    a = pt_bs2 / n_bs2
                    b = gain / (n_bs2 * n_bs1)
                    c = np.sin(math.pi / n_bs2 * (z1 - v1 + gb) * n_bs1)
                    d = np.sin(math.pi / n_bs2 * (z1 - v1 + gb))
                    e = np.absolute((c / d) ** 2)
                    inipower = a*(b * e)

                else:
                    a = pt_bs2 / n_bs2
                    b = gain / (n_bs2 * n_bs1)
                    c = np.sin(math.pi / n_bs1 * (z1 - v1 + gb) * OFDM_overlap * Tn_bs2)
                    d = np.sin(math.pi / n_bs1 * (z1 - v1 + gb))
                    e = np.absolute((c / d) ** 2)

                    g = np.sin(math.pi / n_bs1 * (z1 - v1 - gb)*Tn_bs2)  
                    i = np.absolute((g / d) ** 2)
                    j = OFDM_overlap * i
                    inipower = a*(b * (e + j))     


            # for z1 in range(1, z+1):
            #     for v1 in range(v, 1, -1):
            #         if not z1 == v1:
            #             d = np.sin(math.pi / n_bs2 * (z1 - v1 - gb) * OFDM_overlap * n_bs1)
            #             e = np.sin(math.pi / n_bs2 * (z1 - v1 - gb))
            #             c = np.absolute((d / e) ** 2)
            #             f = OFDM_overlap * c
            #             b = gain / (n_bs2 * n_bs1) * (c + f)
            #             total_v = total_v + b
            #             print(total_v)
            #     total_z = total_z + total_v
            # inipower = a * total_z
            #print('ini Power = ' + str(inipower_b))

        ini_power.append(inipower)
        test2 = np.array(ini_power)
        total1 = np.sum(test2)
        total =  dbW_watts(total1)

        return total1

    def spectrum_efficiency(self):
        SE = np.log2(1 + self.SINR())
        #print('Spectrum Efficiency = ' + str(SE))
        return SE

    def _compute_channel(self, distance):
        # Np is the number of paths p
        PLE_L = 2
        PLE_N = 4
        G_ant = 3 # dBi for beamforming mmWave antennas
        
        # Override the antenna gain if no beamforming
        if self.use_beamforming == False:
            G_ant = self.G_ant_no_beamforming
            
        # theta is the steering angle.  Sampled iid from unif(0,pi).
        theta = np.random.uniform(low=0, high=math.pi, size=self.Np)

        path_loss_LOS = 10 ** (self._path_loss_sub6(distance) / 10.)
        path_loss_NLOS = 10 ** (self._path_loss_sub6(distance) / 10.)
            
        # Bernoulli for p
        alpha = np.zeros(self.Np, dtype=complex)
        p = np.random.binomial(1, self.prob_LOS)
        
        if (p == 1):
            self.Np = 1
            alpha[0] = 1. / math.sqrt(path_loss_LOS)
        else:
            ## just changed alpha to be complex in the case of NLOS
            alpha = (np.random.normal(size=self.Np) + 1j * np.random.normal(size=self.Np)) / math.sqrt(path_loss_NLOS)
                
        rho = 1. * 10 ** (G_ant / 10.)
        
        # initialize the channel as a complex variable.
        h = np.zeros(self.M_ULA, dtype=complex)
        
        for p in np.arange(self.Np):
            a_theta = self._compute_bf_vector(theta[p])
            h += alpha[p] / rho * a_theta.T # scalar multiplication into a vector
        
        h *= math.sqrt(self.M_ULA)
        
#        print ('Warning: channel gain is {} dB.'.format(10*np.log10(LA.norm(h, ord=2))))
        return h        

    def _path_loss_sub6(self, distance):
        f_c = self.f_c
        c = 3e8 # speed of light
        d = distance
        h_B = 20
        h_R = 1.5

#        print('Distance from cell site is: {} km'.format(d/1000.))
        # FSPL
        L_fspl = -10*np.log10((4.*math.pi*c/f_c / d) ** 2)

        
        # COST231        
        C = 3
        a = (1.1 * np.log10(f_c/1e6) - 0.7)*h_R - (1.56*np.log10(f_c/1e6) - 0.8)
        L_cost231  = 46.3 + 33.9 * np.log10(f_c/1e6) + 13.82 * np.log10(h_B) - a + (44.9 - 6.55 * np.log10(h_B)) * np.log10(d/1000.) + C
    
        L = L_cost231
        
        return L # in dB

    def _compute_bf_vector(self, theta):
        c = 3e8 # speed of light
        wavelength = c / self.f_c
        
        d = wavelength / 2. # antenna spacing 
        k = 2. * math.pi / wavelength
    
        exponent = 1j * k * d * math.cos(theta) * np.arange(self.M_ULA)
        
        f = 1. / math.sqrt(self.M_ULA) * np.exp(exponent)
        
        # Test the norm square... is it equal to unity? YES.
    #    norm_f_sq = LA.norm(f, ord=2) ** 2
     #   print(norm_f_sq)
    
        return f   
    
    def minpower(self,numUE,distances):
        distance = max(distances)
        Po = -121
        gu = 1
        gn = 1
        miu = 8
        Mu = 1200*0.8/200*numUE
        #pathloss = 16.8+33.2*np.log10(distance)
        pathloss2 = self._path_loss_sub6(distance)
        Pu= Po - gu - gn + miu + pathloss2 + 10*np.log10(Mu)
        
        Pu = dbW_watts(Pu-30)

        return Pu   

env=environment()
        

In [90]:
# dbm = 46
# watt = 0 
# watt0 = 40

# db = dbm_dbW(dbm)
# watt = dbW_watts(db)
# print(watt)

# dbw = watts_dbW(watt0)
# dbm0 = dbW_dbm (dbw)
# print(dbm0)




In [91]:
env=environment()

In [92]:
env.observation_space.sample()

array([13.915954  , 19.952623  ,  0.7441781 ,  0.5566945 ,  0.84379524,
       93.87628   , 45.        ,  5.        ,  3.795291  ,  3.3747659 ,
        1.557662  ,  1.7865388 ,  3.7374797 ], dtype=float32)

In [93]:
env.reset()

array([6.33550633e+01, 1.45876024e+01, 1.09418984e-01, 7.24562131e-01,
       5.28805323e-01, 1.75682937e+02, 4.50000000e+01, 1.89463411e+00,
       3.30945753e-01, 1.14942931e-01, 1.24107459e+00, 2.22972750e+00,
       1.29256507e+00])

## 3. Test Environment

In [94]:
# episodes = 1
# for episode in range(1, episodes+1):
#     state = env.reset()
#     print(state)
#     done = False
#     score = 0 
    
#     while not done:
#         env.render()
#         action = env.action_space.sample()
#         n_state, reward, done, info = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{}'.format(episode, score))
#     print('Updated State:{}'.format(n_state))
# env.close()

In [95]:
# env.close()

# 4. Train Model

In [96]:
#log_path = os.path.join('Training', 'Logs')

In [97]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=logdir)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [98]:
total_timesteps = 10000
for i in range(90):
    model.learn(total_timesteps, reset_num_timesteps=False, tb_log_name="PPO")
    model.save(f"{models_dir}/{total_timesteps*i}")
model.save('PPO32')  

Logging to ML Rl 5g32/logs\PPO_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | 1.97e+05 |
| time/              |          |
|    fps             | 10       |
|    iterations      | 1        |
|    time_elapsed    | 195      |
|    total_timesteps | 2048     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 100           |
|    ep_rew_mean          | -5.3e+04      |
| time/                   |               |
|    fps                  | 9             |
|    iterations           | 2             |
|    time_elapsed         | 417           |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 3.0559022e-09 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.24         |
|    explai

# 5. Save Model

In [99]:
# tensorboard --logdir=logs

In [100]:
# model.save('PPO')

In [101]:
# from google.colab import drive
# drive.mount('/content/drive')

In [102]:
# evaluate_policy(model, env, n_eval_episodes=10, render=False)

### 6. Bash Terminal

In [103]:
# !pip install google-colab-shell
# from google_colab_shell import getshell
# getshell()

In [ ]:
def test(self):
    UE = 15.69694
    UEpwr = 13
    ULDL = 0.5
    ULDL2 = 0.5
    ULDL3 = 0.5
    d = 225
    tx = 35
    intP = 20
    g1 = 0
    g2 = 0
    s1 = 1
    s2 = 1
    s3 = 1


    li = [UE,UEpwr,ULDL,ULDL2,ULDL3,d,tx,intP,g1,g2,s1,s2,s3,]
    variable = np.array(li)
    print(variable)
    return li

In [177]:
model_path = f"{models_dir}/100000.zip"
model = PPO.load(model_path, env=env)

episodes = 1

for ep in range(episodes):
    obs = env.reset() #variable
    print(obs)
    high_reward = 0
    done = False
    while not done:
        action, _states = model.predict(obs)
        obs, rewards, done, info = env.step(action)
        current_reward = rewards
        #print(current_reward)
        if high_reward < current_reward:
            high_reward = current_reward
            print(obs)
            print(rewards)
        else:
            high_reward = high_reward
    print(obs)        
    #check rewards for used link


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[ 98.47698973  13.70654222   0.8755795    0.26890065   0.78502535
 127.41381726  45.           1.51977286   3.23259913   3.39543613
   2.27696008   1.10401475   1.46558212]
Ue_mmtc_UL = 4
Ue_EMBB_UL = 18
Ue_URLLC_UL = 29
Ue_mmtc_DL = 3
Ue_EMBB_DL = 16
Ue_URLLC_DL = 29


KeyboardInterrupt: 